In [83]:
import pymongo
from datetime import datetime, timedelta
%matplotlib inline
import matplotlib.pyplot as plt
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import string
from dateutil.relativedelta import relativedelta

client = pymongo.MongoClient('localhost',27017)
mydb = client['project3']
mycol = mydb['COVID19']

def display_country():
    country = []
    for document in mycol.distinct("Country"):
        country.append(document)
    return country

def lister(x):
    l =[]
    for i in range(1,x+1):
        l.append(i)
    return l

def display_WHO():
    who = []
    for document1 in mycol.distinct("WHO_Region"):
        who.append(document1)
    return who

def display_letter():
    letter = list(string.ascii_uppercase)
    return letter

country = display_country()
month = lister(12)
date = lister(31)
who = display_WHO()
alpha = display_letter()

In [77]:
def query1(country,month,date):    # Count of new cases
    x = []
    y = []
    date1 = ""
    date2 = ""
    if int(month) < 10:
        date1 = "2020-0"+ str(month)
        date2 = "2020-0"+ str(month)
    else:
        date1 = "2020-"+ str(month)
        date2 = "2020-"+ str(month)
        
    if int(date) < 10:
        date1 = date1 + "-0" + str(date)
    else :
        date1 = date1 + "-" + str(date)
    date = int(date)+10
    date2 = date2 + "-" + str(date)
    query = { "Country": country, "Date": {"$gt": date1, "$lte": date2}}
    result = mycol.find(query,{ "_id":0, "Date":1,"New_Cases":1,})
    for l in result:
        x.append(l["Date"])
        y.append(l["New_Cases"])
    plt.rcParams['figure.figsize'] = (20.0, 10.0)
    plt.plot(x, y) 

interactive_plot = interactive(query1, country = widgets.Dropdown(options = country, value = "Afghanistan"),
                                       month = widgets.Dropdown(options = month, value = 1),
                                       date = widgets.Dropdown(options = date, value = 1))
output = interactive_plot.children[-1]
output.layout.height = '500px'
display(interactive_plot)

interactive(children=(Dropdown(description='country', options=('Afghanistan', 'Albania', 'Algeria', 'American …

In [78]:
def query2(letter,month):   # Maximum new cases and new deaths count
    l_s = "^" + letter
    x = []
    y1 = []
    y2 = []
    date1 = ""
    date2 = ""
    date = 1
    if int(month) < 10:
        date1 = "2020-0"+ str(month)
        date2 = "2020-0"+ str(month)
    else:
        date1 = "2020-"+ str(month)
        date2 = "2020-"+ str(month)
        
    if int(date) < 10:
        date1 = date1 + "-0" + str(date)
    else :
        date1 = date1 + "-" + str(date)
    date = int(date)+30
    date2 = date2 + "-" + str(date)
    query = [    {"$match" : {"Country": {"$regex": l_s}, "Date":{"$gt": date1, "$lte": date2}}},
                 {"$group": { "_id": "$Country",
                             "Max_d": {"$max":"$New_Deaths"},
                             "Max_c": {"$max":"$New_Cases"}}}
            ]
    result = mycol.aggregate(query)
    for l in result:
        x.append(l["_id"])
        y1.append(l["Max_d"])
        y2.append(l["Max_c"])
    fig, ax1 = plt.subplots()
    plt.xticks(rotation = 45)
    ax2 = ax1.twinx()
    ax1.plot(x, y1, 'r-')
    ax2.plot(x, y2, 'b-')

    ax1.set_xlabel('Country')
    ax1.set_ylabel('Max_Deaths', color='r')
    ax2.set_ylabel('Max_Cases', color='b')

interactive_plot = interactive(query2,letter = widgets.Dropdown(options = alpha, value = "A"),
                                       month = widgets.Dropdown(options = month, value = 1))
output = interactive_plot.children[-1]
output.layout.height = '550px'
display(interactive_plot)


interactive(children=(Dropdown(description='letter', options=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'…

In [79]:
def query3(letter,month): # Minimum new cases and new deaths count
    l_s = "^" + letter
    x = []
    y1 = []
    y2 = []
    date1 = ""
    date2 = ""
    date = 1
    if int(month) < 10:
        date1 = "2020-0"+ str(month)
        date2 = "2020-0"+ str(month)
    else:
        date1 = "2020-"+ str(month)
        date2 = "2020-"+ str(month)
        
    if int(date) < 10:
        date1 = date1 + "-0" + str(date)
    else :
        date1 = date1 + "-" + str(date)
    date = int(date)+30
    date2 = date2 + "-" + str(date)
    query = [    {"$match" : {"Country": {"$regex": l_s}, "Date":{"$gt": date1, "$lte": date2}}},
                 {"$group": { "_id": "$Country",
                             "min_d": {"$min":"$New_Deaths"},
                             "min_c": {"$min":"$New_Cases"}}}
            ]
    result = mycol.aggregate(query)
    for l in result:
        x.append(l["_id"])
        y1.append(l["min_d"])
        y2.append(l["min_c"])
    fig, ax1 = plt.subplots()
    plt.xticks(rotation = 45)
    ax2 = ax1.twinx()
    ax1.plot(x, y1, 'r-')
    ax2.plot(x, y2, 'b-')

    ax1.set_xlabel('Country')
    ax1.set_ylabel('min_Deaths', color='r')
    ax2.set_ylabel('min_Cases', color='b')

interactive_plot = interactive(query3,letter = widgets.Dropdown(options = alpha, value = "A"),
                                       month = widgets.Dropdown(options = month, value = 1))
output = interactive_plot.children[-1]
output.layout.height = '550px'
display(interactive_plot)



interactive(children=(Dropdown(description='letter', options=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'…

In [80]:
def query4(letter): # Sum of cumulative new cases and new deaths count:
    l = "^" + letter
    x = []
    y1 = []
    y2 = []
    query = [    {"$match" : {"Country": {"$regex": l}}},
                 {"$group":{ "_id": "$Country",
                             "Average":{ "$sum":"$ Cumulative_Cases"},
                             "Average_d":{ "$sum":"$Cumulative_Deaths"}}}
            ]
    result = mycol.aggregate(query)
    for l in result:
        x.append(l["_id"])
        y1.append(l["Average"])
        y2.append(l["Average_d"])
    fig, ax1 = plt.subplots()
    plt.xticks(rotation = 45)
    ax2 = ax1.twinx()
    ax1.plot(x, y1, 'r-')
    ax2.plot(x, y2, 'b-')

    ax1.set_xlabel('Country')
    ax1.set_ylabel('Cumulative_Cases', color='r')
    ax2.set_ylabel('Cumulative_Deaths', color='b')
    
interactive_plot = interactive(query4,letter = widgets.Dropdown(options = alpha, value = "A" ))
output = interactive_plot.children[-1]
output.layout.height = '550px'
interactive_plot

 

interactive(children=(Dropdown(description='letter', options=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'…

In [81]:
def query5(region): # Average of new deaths count
    x = []
    y = []
    query = [    {"$match" : {"WHO_Region": region}},
                 {"$group":{ "_id": "$Country",
                             "Average":{ "$avg": "$New_Deaths"}}}
            ]
    result = mycol.aggregate(query)
    for l in result:
        x.append(l["_id"])
        y.append(l["Average"])
    plt.rcParams['figure.figsize'] = (25.0, 10.0)
    plt.xticks(rotation=45)
    plt.plot(x, y) 

interactive_plot = interactive(query5, region = widgets.Dropdown(options = who, value = "EMRO"))
output = interactive_plot.children[-1]
output.layout.height = '550px'
display(interactive_plot)



interactive(children=(Dropdown(description='region', options=('EMRO', 'EURO', 'AFRO', 'WPRO', 'AMRO', 'SEARO',…

In [82]:
def query6(country, month): # Ratio of new cases count and cumulative cases count AND 
                            # ratio of  new deaths count and cumulative deaths count
    x = []
    y1 = []
    y2 = []
    date = 1
    date1 = ""
    date2 = ""
    if int(month) < 10:
        date1 = "2020-0"+ str(month)
        date2 = "2020-0"+ str(month)
    else:
        date1 = "2020-"+ str(month)
        date2 = "2020-"+ str(month)
        
    if int(date) < 10:
        date1 = date1 + "-0" + str(date)
    else :
        date1 = date1 + "-" + str(date)
    date = int(date)+30
    date2 = date2 + "-" + str(date)
    query = [ {"$match": {"Country":country," Cumulative_Cases": {"$ne": 0}, "Cumulative_Deaths": {"$ne": 0}, "Date": {"$gt": date1, "$lte": date2}}},
              {"$project": {
                            "Date":"$Date",
                            "Ratio_New": { "$divide" : [ "$New_Cases", "$ Cumulative_Cases" ] },
                            "Ratio_Deaths": { "$divide" : [ "$New_Deaths", "$Cumulative_Deaths" ] } }},
            ]
    result = mycol.aggregate(query)
    for l in result:
        x.append(l["Date"])
        y1.append(l["Ratio_New"])
        y2.append(l["Ratio_Deaths"])
    fig, ax1 = plt.subplots()
    plt.xticks(rotation = 45)
    ax2 = ax1.twinx()
    ax1.plot(x, y1, 'r-')
    ax2.plot(x, y2, 'b-')

    ax1.set_xlabel('Date')
    ax1.set_ylabel('Ratio_Cases', color='r')
    ax2.set_ylabel('Ratio_Deaths', color='b')
 
interactive_plot = interactive(query6, country = widgets.Dropdown(options = country, value = "Afghanistan"),
                                       month = widgets.Dropdown(options = month, value = 1))
output = interactive_plot.children[-1]
output.layout.height = '550px'
display(interactive_plot) 
    
    

interactive(children=(Dropdown(description='country', options=('Afghanistan', 'Albania', 'Algeria', 'American …